# 🌌 Spectral Affinity Master: Ultimate Suno Master Pipeline (v6.0)

**6-Stage Professional Restoration** specifically engineered for AI-generated audio (Suno, Udio, etc.)  
combined with **Neural Semantic Affinity** for intelligent track organization.

### 🚀 The Pipeline:
| # | Stage | What It Does | Speed |
|---|-------|-------------|-------|
| 1 | 🧹 **Neural Cleaning** (DeepFilterNet 3) | AI artifact & noise removal | ~2s/track |
| 2 | 🔊 **Mono-Bass Phase Correction** | Solid low-end (< 150Hz → mono) | instant |
| 3 | 💥 **Transient Re-synthesis (Punch)** | Restore dynamics & attack energy | instant |
| 4 | ✨ **Spectre Restoration** | Multi-band harmonic exciter (48kHz) | ~1s/track |
| 5 | 🏝️ **Affinity Grouping** (MERT + K-Means) | Neural semantic clustering + Camelot | batch |
| 6 | 🎚️ **Mastering Match** (Matchering) | Reference-based loudness & tone | ~3s/track |

### 1. 🛠️ Ultimate Environment Setup

In [ ]:
import os, warnings, json, torch, torchaudio, librosa, glob, shutil, re, gc, pathlib
import numpy as np
from tqdm.auto import tqdm
from concurrent.futures import ThreadPoolExecutor
from IPython.display import HTML, FileLink, display
import torchaudio.transforms as T
import torchaudio.functional as F
from transformers import Wav2Vec2FeatureExtractor, AutoModel, logging as hf_logging
from sklearn.preprocessing import normalize
from sklearn.cluster import KMeans as skKMeans

warnings.filterwarnings('ignore')
hf_logging.set_verbosity_error()
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🔥 ACCELERATOR: {torch.cuda.get_device_name(0) if device == 'cuda' else 'CPU'}")

# ── Core DSP ──
try:
    from nnAudio.Spectrogram import CQT1992v2
except:
    !pip install -q nnAudio transformers
    from nnAudio.Spectrogram import CQT1992v2

# ── DeepFilterNet 3 ──
try:
    from df.enhance import init_df, enhance as df_enhance
    HAS_DFN = True
except:
    !pip install -q deepfilternet
    try:
        from df.enhance import init_df, enhance as df_enhance
        HAS_DFN = True
    except:
        HAS_DFN = False
print(f"{'✅' if HAS_DFN else '⚠️'} DeepFilterNet 3: {'ready' if HAS_DFN else 'unavailable (skipping neural clean)'}")

# ── Matchering ──
try:
    import matchering as mg
    HAS_MATCHERING = True
except:
    !pip install -q matchering
    try:
        import matchering as mg
        HAS_MATCHERING = True
    except:
        HAS_MATCHERING = False
print(f"{'✅' if HAS_MATCHERING else '⚠️'} Matchering: {'ready' if HAS_MATCHERING else 'unavailable (skipping mastering match)'}")

# ── cuML (optional GPU clustering) ──
try:
    from cuml.cluster import KMeans as cuKMeans
    HAS_CUML = True
except:
    HAS_CUML = False

### 2. 🧠 Neural Analysis Engine (MERT + DSP)

In [ ]:
class SpectralMasterEngine:
    def __init__(self, device='cuda', sr=24000, cache_file='spectral_master_cache.json'):
        self.device = device
        self.sr = sr
        self.cache_file = cache_file
        self.cache = self._load_cache()
        
        self.cqt_layer = CQT1992v2(sr=self.sr, n_bins=84, bins_per_octave=12).to(self.device)
        major = torch.tensor([6.35, 2.23, 3.48, 2.33, 4.38, 4.09, 2.52, 5.19, 2.39, 3.66, 2.29, 2.88], device=device)
        minor = torch.tensor([6.33, 2.68, 3.52, 5.38, 2.60, 3.53, 2.54, 4.75, 3.98, 2.69, 3.34, 3.17], device=device)
        self.profiles = torch.stack([torch.roll(major, i) for i in range(12)] + 
                                    [torch.roll(minor, i) for i in range(12)]).t()
        
        print("🧠 Loading MERT Neural Brain...")
        self.mert_model_id = "m-a-p/MERT-v1-95M"
        self.processor = Wav2Vec2FeatureExtractor.from_pretrained(self.mert_model_id, trust_remote_code=True)
        self.mert_model = AutoModel.from_pretrained(self.mert_model_id, trust_remote_code=True).to(self.device)
        self.mert_model.eval()

    def _load_cache(self):
        if os.path.exists(self.cache_file):
            with open(self.cache_file, 'r') as f: return json.load(f)
        return {}

    def save_cache(self):
        with open(self.cache_file, 'w') as f: json.dump(self.cache, f)

    def get_camelot(self, key, mode):
        c_map = {
            ('B', 'major'): '01B', ('F#', 'major'): '02B', ('C#', 'major'): '03B', ('G#', 'major'): '04B',
            ('D#', 'major'): '05B', ('A#', 'major'): '06B', ('F', 'major'): '07B', ('C', 'major'): '08B',
            ('G', 'major'): '09B', ('D', 'major'): '10B', ('A', 'major'): '11B', ('E', 'major'): '12B',
            ('G#', 'minor'): '01A', ('D#', 'minor'): '02A', ('A#', 'minor'): '03A', ('F', 'minor'): '04A',
            ('C', 'minor'): '05A', ('G', 'minor'): '06A', ('D', 'minor'): '07A', ('A', 'minor'): '08A',
            ('E', 'minor'): '09A', ('B', 'minor'): '10A', ('F#', 'minor'): '11A', ('C#', 'minor'): '12A'
        }
        return c_map.get((key, mode.lower()), "00X")

    def process_batch(self, batch_data):
        paths = [x[0] for x in batch_data]
        audios = [x[1] for x in batch_data]
        durations = [x[2] for x in batch_data]
        dsp_tensor = torch.zeros(len(audios), self.sr * 120, device=self.device)
        for i, a in enumerate(audios):
            l = min(len(a), self.sr * 120)
            dsp_tensor[i, :l] = torch.from_numpy(a[:l]).to(self.device)
        
        with torch.no_grad():
            spec = self.cqt_layer(dsp_tensor)
            energy = spec.pow(2).mean(dim=(1, 2)).cpu().numpy()
            chroma = spec.view(len(audios), 7, 12, -1).sum(dim=(1, 3))
            chroma = chroma / (chroma.norm(dim=1, keepdim=True) + 1e-6)
            corrs = torch.matmul(chroma, self.profiles)
            best_idx = torch.argmax(corrs, dim=1).cpu().numpy()
            
            embeddings_list = []
            pc = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
            for i in range(len(audios)):
                mid = len(audios[i])//2
                slice_len = int(self.sr * 15)
                audio_slice = audios[i][mid : mid + slice_len] if len(audios[i]) > slice_len else audios[i]
                input_values = self.processor(audio_slice, sampling_rate=self.sr, return_tensors="pt").input_values.to(self.device)
                emb = self.mert_model(input_values).last_hidden_state.mean(dim=1).squeeze().cpu().numpy()
                embeddings_list.append(emb.tolist())
                
        results = []
        for i in range(len(audios)):
            res = {
                'key': pc[best_idx[i] % 12], 'mode': 'major' if best_idx[i] < 12 else 'minor',
                'energy': float(energy[i]), 'duration': float(durations[i]), 
                'embedding': embeddings_list[i], 'path': paths[i]
            }
            try:
                onset = librosa.onset.onset_strength(y=audios[i][:self.sr*60], sr=self.sr)
                res['bpm'] = float(librosa.beat.tempo(onset_envelope=onset, sr=self.sr, aggregate=np.mean)[0])
            except: res['bpm'] = 120.0
            res['camelot'] = self.get_camelot(res['key'], res['mode'])
            results.append(res)
        return results

### 3. 💎 Ultimate Suno Master: 6-Stage Restoration Pipeline

| Stage | Technology | Purpose |
|-------|-----------|----------|
| 🧹 Neural Clean | DeepFilterNet 3 (AI) | Remove compression artifacts, digital fizz |
| 🔊 Mono-Bass | Linkwitz-Riley Crossover | Phase-coherent solid low-end |
| 💥 Transient Punch | Envelope Follower + Gain Mask | Restore dynamics Suno crushes |
| ✨ Spectre Restore | Multi-band Harmonic Exciter | Recover lost high frequencies |
| 🎚️ Mastering Match | Matchering (DSP) | Match loudness & tone to reference |

In [ ]:
class UltimateSunoMaster:
    """4-Stage GPU-accelerated restoration for AI-generated audio."""

    def __init__(self, device='cuda', target_sr=48000, stages=None):
        self.device = device
        self.target_sr = target_sr
        self.stages = stages or {'neural_clean': True, 'mono_bass': True, 'transient_punch': True, 'spectre_restore': True}
        self.dfn_available = False
        if self.stages.get('neural_clean') and HAS_DFN:
            try:
                self._dfn_model, self._df_state, _ = init_df()
                self.dfn_available = True
                print('  ✅ DeepFilterNet 3 model loaded')
            except Exception as e:
                print(f'  ⚠️ DFN3 init failed: {e}')

    def _to_48k(self, wav, sr):
        if sr != self.target_sr:
            return T.Resample(sr, self.target_sr).to(self.device)(wav)
        return wav

    # ── Stage 1: Neural Cleaning (DeepFilterNet 3) ──
    def neural_clean(self, wav):
        if not self.dfn_available: return wav
        try:
            return df_enhance(self._dfn_model, self._df_state, wav, atten_lim_db=6)
        except: return wav

    # ── Stage 2: Mono-Bass Phase Correction ──
    def mono_bass(self, wav, cutoff=150):
        sr = self.target_sr
        low = F.lowpass_biquad(wav, sr, cutoff)
        low = F.lowpass_biquad(low, sr, cutoff)  # Linkwitz-Riley
        high = wav - low
        if wav.shape[0] >= 2:
            low = low.mean(dim=0, keepdim=True).expand_as(low)
        return low + high

    # ── Stage 3: Transient Re-synthesis (Punch) ──
    def transient_punch(self, wav, boost_db=4.0, release_ms=25):
        sr = self.target_sr
        mono = wav.mean(dim=0) if wav.shape[0] >= 2 else wav.squeeze(0)
        frame_len = int(sr * 0.005)  # 5ms frames
        hop = frame_len // 2
        if frame_len < 2: return wav

        padded = torch.nn.functional.pad(mono, (frame_len//2, frame_len//2))
        energy = padded.unfold(0, frame_len, hop).pow(2).mean(dim=-1).sqrt()

        # Positive spectral flux = onset detection
        flux = torch.clamp(torch.diff(energy, prepend=energy[:1]), min=0)
        if flux.max() < 1e-8: return wav
        flux_n = flux / (flux.max() + 1e-8)
        thr = flux_n.mean() + 1.5 * flux_n.std()
        mask = torch.clamp((flux_n - thr) / (1.0 - thr + 1e-8), 0, 1)

        # Upsample to sample resolution
        gain = torch.nn.functional.interpolate(
            mask[None, None, :], size=wav.shape[-1], mode='linear', align_corners=False
        ).squeeze()

        # Release smoothing kernel
        rel = max(int(sr * release_ms / 1000), 4)
        k = torch.exp(-torch.arange(rel, device=self.device, dtype=torch.float32) / (rel/4))
        k = (k / k.sum())[None, None, :]
        gain = torch.nn.functional.conv1d(gain[None, None, :], k, padding=rel//2).squeeze()[:wav.shape[-1]]

        # Apply transient boost
        boost = 10 ** (boost_db / 20)
        result = wav * (1.0 + gain.unsqueeze(0) * (boost - 1.0))
        peak = result.abs().max()
        return result * (0.98 / peak) if peak > 0.98 else result

    # ── Stage 4: Spectre High-End Restoration (Multi-Band Exciter) ──
    def spectre_restore(self, wav):
        sr = self.target_sr
        stft = torch.stft(wav[0], n_fft=4096, hop_length=1024,
                          window=torch.hann_window(4096).to(self.device), return_complex=True)
        mag_db = 20 * torch.log10(torch.abs(stft).mean(dim=1) + 1e-8)
        freqs = torch.linspace(0, sr/2, mag_db.shape[0]).to(self.device)
        mask = mag_db > (mag_db.max() - 55)
        cutoff = freqs[mask][-1].item() if mask.any() else 16000.0
        cutoff = max(12000.0, min(cutoff, 22000.0))
        if cutoff > 20000: return wav

        # Band 1: Presence exciter
        exc1 = F.highpass_biquad(torch.tanh(F.highpass_biquad(wav, sr, cutoff*0.85) * 1.8), sr, cutoff*0.9)
        # Band 2: Air exciter
        exc2 = F.highpass_biquad(torch.tanh(F.highpass_biquad(wav, sr, cutoff) * 3.0), sr, cutoff)

        y = wav + (exc1 * 0.08) + (exc2 * 0.15)
        peak = y.abs().max()
        return y * (0.98 / peak) if peak > 0.98 else y

    # ── Full Pipeline ──
    def process_track(self, input_path, output_path, verbose=False):
        try:
            wav, sr = torchaudio.load(input_path)
            wav = self._to_48k(wav.to(self.device), sr)
            if self.stages.get('neural_clean'):  wav = self.neural_clean(wav)
            if self.stages.get('mono_bass'):     wav = self.mono_bass(wav)
            if self.stages.get('transient_punch'): wav = self.transient_punch(wav)
            if self.stages.get('spectre_restore'): wav = self.spectre_restore(wav)
            torchaudio.save(output_path, wav.cpu(), self.target_sr, encoding='PCM_S', bits_per_sample=16)
            return True
        except Exception as e:
            print(f'  ⚠️ FAILED {os.path.basename(input_path)}: {e}')
            return False


class MasteringEngine:
    """Stage 6: Reference-based mastering via Matchering."""

    def __init__(self, reference_path=None):
        self.ref = reference_path
        self.available = HAS_MATCHERING and reference_path and os.path.exists(str(reference_path))
        if self.available:
            print(f'  🎚️ Mastering Engine ready | Ref: {os.path.basename(reference_path)}')
        elif reference_path:
            print(f'  ⚠️ Reference not found: {reference_path}')
        else:
            print('  ℹ️  Mastering Match disabled (set REFERENCE_TRACK to enable)')

    def master(self, input_path, output_path):
        if not self.available:
            if input_path != output_path: shutil.copy2(input_path, output_path)
            return False
        try:
            mg.process(target=input_path, reference=self.ref, results=[mg.pcm16(output_path)])
            return True
        except Exception as e:
            print(f'  ⚠️ Mastering error: {e}')
            if input_path != output_path: shutil.copy2(input_path, output_path)
            return False

### 4. 🌈 Chromatic Flow & Affinity Logic

In [ ]:
def get_next_harmonic(current_cam):
    num, alpha = int(current_cam[:2]), current_cam[2]
    return [
        f"{str(num).zfill(2)}{alpha}",
        f"{str((num % 12) + 1).zfill(2)}{alpha}",
        f"{str(((num - 2) % 12) + 1).zfill(2)}{alpha}",
        f"{str(num).zfill(2)}{'A' if alpha == 'B' else 'B'}"
    ]

def sequence_chromatic_set(tracks, target_duration):
    if not tracks: return []
    pool = list(tracks)
    current = pool.pop(0)
    ordered_set = [current]
    current_dur = current['duration']
    
    while pool and current_dur < target_duration:
        compat_keys = get_next_harmonic(current['camelot'])
        def score(t):
            h_score = 1.0 if t['camelot'] in compat_keys else (0.8 if t['camelot'] == current['camelot'] else 0.0)
            bpm_diff = abs(t['bpm'] - current['bpm'])
            b_score = max(0, 1.0 - (bpm_diff / 40.0))
            s_score = np.dot(current['embedding'], t['embedding']) / (np.linalg.norm(current['embedding']) * np.linalg.norm(t['embedding']) + 1e-9)
            return (h_score * 0.5) + (s_score * 0.3) + (b_score * 0.2)
        
        pool.sort(key=score, reverse=True)
        next_t = pool.pop(0)
        ordered_set.append(next_t)
        current_dur += next_t['duration']
        current = next_t
    return ordered_set

def clean_name(n):
    n = os.path.basename(n).rsplit('.', 1)[0]
    n = re.sub(r"^[\w\-]+?-", "", n)
    return re.sub(r"[\-\_\.]+?", " ", n).strip()

### 5. 🚀 Execution Pipeline

In [ ]:
# ╔══════════════════════════════════════════════════════════════╗
# ║  CONFIG                                                      ║
# ╚══════════════════════════════════════════════════════════════╝
INPUT_DIR       = "/kaggle/input/datasets/danieldobles/slavic-set"
OUTPUT_DIR      = "/kaggle/working/master_organized"
TEMP_DIR        = "/kaggle/working/_temp_restore"
BATCH_SIZE      = 16
SET_DUR         = 75 * 60   # 75 min sets
N_CLUSTERS      = 3

# ── Restoration Config ──
REFERENCE_TRACK = ""   # Path to a commercial reference for mastering match
STAGES = {
    'neural_clean':    True,   # Stage 1: DeepFilterNet 3
    'mono_bass':       True,   # Stage 2: Sub-bass to mono (< 150Hz)
    'transient_punch': True,   # Stage 3: Restore attack dynamics
    'spectre_restore': True,   # Stage 4: High-end harmonic exciter
    'matchering':      True,   # Stage 6: Reference-based mastering
}
PUNCH_BOOST_DB  = 4.0   # Transient boost (2-6 dB recommended)
BASS_CUTOFF_HZ  = 150   # Mono-bass crossover frequency

# ╔══════════════════════════════════════════════════════════════╗
# ║  INIT ENGINES                                                ║
# ╚══════════════════════════════════════════════════════════════╝
print('\n🔧 Initializing engines...')
analyzer  = SpectralMasterEngine(device=device, sr=24000)
restorer  = UltimateSunoMaster(device=device, stages=STAGES)
mastering = MasteringEngine(reference_path=REFERENCE_TRACK) if STAGES.get('matchering') else None

# ╔══════════════════════════════════════════════════════════════╗
# ║  PHASE 1: SCAN & ANALYZE                                     ║
# ╚══════════════════════════════════════════════════════════════╝
print('\n🔍 Scanning Library...')
paths = []
for ext in ['*.mp3', '*.wav', '*.flac', '*.m4a']:
    paths.extend(glob.glob(os.path.join(INPUT_DIR, '**', ext), recursive=True))
paths = sorted(set(paths))
print(f'   Found {len(paths)} tracks')

to_analyze = [p for p in paths if p not in analyzer.cache]
if to_analyze:
    chunks = [to_analyze[i:i+BATCH_SIZE] for i in range(0, len(to_analyze), BATCH_SIZE)]
    with ThreadPoolExecutor(max_workers=2) as pool:
        for chunk in tqdm(chunks, desc='🔥 AI Analysis'):
            def load(p):
                try: y, _ = librosa.load(p, sr=analyzer.sr); return (p, y, len(y)/analyzer.sr)
                except: return None
            batch_data = [x for x in list(pool.map(load, chunk)) if x is not None]
            results = analyzer.process_batch(batch_data)
            for r in results: analyzer.cache[r['path']] = r
            analyzer.save_cache(); gc.collect()

# ╔══════════════════════════════════════════════════════════════╗
# ║  PHASE 2: CLUSTER & SEQUENCE                                 ║
# ╚══════════════════════════════════════════════════════════════╝
library = [analyzer.cache[p] for p in paths if p in analyzer.cache]
if not library:
    print('❌ ERROR: No audio files processed.')
else:
    X = normalize(np.array([t['embedding'] for t in library]))
    KM = cuKMeans(n_clusters=N_CLUSTERS) if HAS_CUML else skKMeans(n_clusters=N_CLUSTERS, n_init=10)
    p_labels = KM.fit_predict(X)

    clusters = {i: [] for i in range(N_CLUSTERS)}
    for i, l in enumerate(p_labels): clusters[l].append(library[i])

    if os.path.exists(OUTPUT_DIR): shutil.rmtree(OUTPUT_DIR)
    os.makedirs(TEMP_DIR, exist_ok=True)

    # ╔══════════════════════════════════════════════════════════╗
    # ║  PHASE 3: RESTORE, SEQUENCE & MASTER                     ║
    # ╚══════════════════════════════════════════════════════════╝
    print('\n🌈 Processing Sets...')
    total_sets, total_tracks, failed = 0, 0, 0

    for c_idx, cluster_tracks in clusters.items():
        c_name = f'Group_{chr(65+c_idx)}'
        temp_pool = sorted(cluster_tracks, key=lambda x: x['energy'])
        set_idx = 1

        while temp_pool:
            ordered_set = sequence_chromatic_set(temp_pool, SET_DUR)
            if not ordered_set: break
            temp_pool = [t for t in temp_pool if t['path'] not in {s['path'] for s in ordered_set}]

            s_dir = os.path.join(OUTPUT_DIR, c_name, f'Set_{set_idx}')
            os.makedirs(s_dir, exist_ok=True)

            for i, t in enumerate(tqdm(ordered_set, desc=f'💎 {c_name} Set {set_idx}', leave=False)):
                meta = f"[{t['camelot']} - {int(t['bpm'])}BPM]"
                out_name = f"{str(i+1).zfill(2)} - {meta} {clean_name(t['path'])}.flac"
                final_path = os.path.join(s_dir, out_name)
                temp_path  = os.path.join(TEMP_DIR, f'temp_{c_idx}_{set_idx}_{i}.wav')

                # Stages 1-4: Restoration
                ok = restorer.process_track(t['path'], temp_path)

                if ok and mastering and mastering.available:
                    # Stage 6: Mastering Match
                    mastering.master(temp_path, final_path)
                    os.remove(temp_path)
                elif ok:
                    shutil.move(temp_path, final_path)
                else:
                    shutil.copy2(t['path'], final_path)
                    failed += 1

                total_tracks += 1
                gc.collect(); torch.cuda.empty_cache()

            set_idx += 1; total_sets += 1

    # Cleanup temp
    if os.path.exists(TEMP_DIR): shutil.rmtree(TEMP_DIR)

    # ╔══════════════════════════════════════════════════════════╗
    # ║  DONE                                                    ║
    # ╚══════════════════════════════════════════════════════════╝
    stages_used = [k for k, v in STAGES.items() if v]
    print(f'\n✅ SUCCESS! {total_tracks} tracks processed ({failed} failed) → {total_sets} sets')
    print(f'   Stages applied: {" → ".join(stages_used)}')
    zip_name = 'SpectralAffinity_UltimateMaster.zip'
    !zip -0 -rq {zip_name} master_organized
    display(HTML(f"<h3>🚀 <a href='{zip_name}' id='dl'>DOWNLOAD ULTIMATE MASTER MIXES</a></h3>"))
    display(HTML("<script>setTimeout(() => document.getElementById('dl').click(), 1000);</script>"))